In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from Points import Points
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import chain

In [16]:
emotion_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,"sad": 6,
            "surprise": 6}

def read_data(path, label_points):
    '''Path for json with '''
    landmarks = pd.read_json(path)
    emotions = np.asarray([emotion_dict[y] for y in landmarks.iloc[0].astype(object)], dtype=np.float32)
    columns = landmarks.shape[1]
    points = []
    for photo in range(columns):
        for point in label_points:
            coords = landmarks[photo].iloc[point+1][0:2]
            points.extend(coords)

    points = np.array(points, dtype=np.float32).reshape(1133, len(label_points*2))

    print(f"shape at the end: {points.shape}")
    return emotions, points
    #return [[a, b] for (a, b) in zip(np.concatenate((JAFFE_EMO, CK_EMO)), np.concatenate((JAFFE_VEC, CK_VEC)))] #
    
def select_points(emotions, points, training_ratio):
    X_train, X_test, y_train, y_test = train_test_split(points, emotions, 
                                                        test_size=(1-training_ratio), random_state=1234)
    
    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
    
    new_y_train = np.zeros([y_train.shape[0], 7])
    for index in range(y_train.shape[0]):
        if int(y_train[index]) != emotion_dict["neutral"]:
            new_y_train[index][int(y_train[index])] = 10
    
    new_y_test = np.zeros([y_test.shape[0], 7])
    for index in range(y_test.shape[0]):
        if int(y_test[index]) != emotion_dict["neutral"]:
            new_y_test[index][int(y_test[index])] = 10
        
    
    y_train = torch.from_numpy(new_y_train)
    y_test = torch.from_numpy(new_y_test)
    
    
    return X_train, X_test, y_train, y_test
    

In [14]:
# Neural Net model

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(n_input_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 7)

    def forward(self, x):
        x = x.view(-1, 268)
        x = F.relu(self.fc1(x), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        x = self.fc3(x)
        return x

model = Model(268)

In [17]:
# 1)
min_wanted_points = Points.right_eye_middle.value + Points.left_eye_middle.value + Points.nose.value + Points.mouth_inner.value
max_wanted_points = min_wanted_points + Points.right_eye_inner.value + Points.right_eye_outer.value + \
        Points.left_eye_inner.value + Points.left_eye_outer.value

emotions, points = read_data("./photo_landmark_list.json", max_wanted_points)
X_train, X_test, y_train, y_test = select_points(emotions, points, .95)

# 2) Loss and optimizer
num_epochs = 1000
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 3) Training loop
counter = 0
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    new_label = torch.from_numpy(y_train).float().view(1076,7)
    loss = criterion(new_pred, new_label)

    # Backward pass and update
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

    with torch.no_grad():
        correct = 0
        y_predicted = model(X_test)
        for row in range(y_predicted.shape[0]):
            dist = 100
            closest_index = None
            for index in range(7):
                if abs(float(y_predicted[row][index])-10) < dist:
                    dist = abs(float(y_predicted[row][index])-10)
                    closest_index = index
            if closest_index == y_test[row]:
                correct += 1

        print(f"correct: {correct}")
        print(f"total: {y_predicted.shape[0]}")

#torch.save(model.state_dict(), "./model-v10.pt")

shape at the end: (1133, 268)


TypeError: expected np.ndarray (got Tensor)